FB page

In [297]:
from bs4 import BeautifulSoup

# Load the HTML file
with open(r'D:\projects\jekyllCM\kucmc_website\tools\FBpageOrg.html', 'r') as file:
    html_content = file.read()

# Parse the HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Find all post containers
post_containers = soup.find_all('div', class_='x1iorvi4 x1pi30zi x1l90r2v x1swvt13')

print(len(post_containers))
# Extract post content
for container in post_containers[10:13]:
    actualPostCont = container.parent.parent
    Allposts =[]
    try:
        postDetail=[]
        # print(container)
        # print(container.parent.parent.find("use").get("xlink:href")[1:])
    
        dates = soup.find("text",id=container.parent.parent.find_all("use")[1].get("xlink:href")[1:])
        print(dates.text)   

        post_text = container.find_all('div')[0].find('div',class_="xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a").text.split("\n")
        captionText=""
        for i in post_text:
            captionText+=i.strip()+" "
        print(captionText)
        postDetail.append(captionText)


        images = actualPostCont.find_all("div",class_="x6ikm8r x10wlt62 x10l6tqk")
        imgLinks =[]
        for img in images:
            imgLinks.append(img.find("img").get("src"))
        if imgLinks:
            Allposts.append(postDetail)
    except:
        print("idk") 
    print('---')
    break


93
idk
---


# Valid Date Maker

In [ ]:
from datetime import datetime

def convert_dates(date_strings):
    converted_dates = []
    current_year = datetime.now().year

    for date_str in date_strings:
        try:
            date_obj = datetime.strptime(date_str, "%B %d")
        except ValueError:
            date_obj = datetime.strptime(date_str, "%B %d, %Y")
        date_obj = date_obj.replace(year=current_year)
        converted_dates.append(date_obj.strftime("%Y-%m-%d"))

    return converted_dates

# Filename Validator

In [ ]:
import re

def sanitize_folder_name(name):
    # Remove any characters that are not letters, numbers, or spaces
    sanitized_name = re.sub(r'[^\w\s]', '', name)
    # Remove leading/trailing spaces and replace internal spaces with underscores
    sanitized_name = sanitized_name.strip().replace(' ', '_')
    return sanitized_name


# Title Maker

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def generate_title_from_paragraph(paragraph):
    # Initialize the parser and tokenizer
    parser = PlaintextParser.from_string(paragraph, Tokenizer("english"))

    # Initialize the LexRank summarizer
    summarizer = LexRankSummarizer()

    # Generate the summary (title) from the paragraph
    summary = summarizer(parser.document, 1)

    if summary:
        # Extract the title as a complete sentence
        title = str(summary[0]).strip()
    else:
        # Fallback: Use the first sentence as the title
        sentences = list(parser.document.sentences)
        if sentences:
            title = str(sentences[0]).strip()
        else:
            # If no sentences are present, return an empty string
            title = paragraph

    return title


# Keywords maker

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import string

nltk.download('punkt')
nltk.download('stopwords')

def extract_top_keywords(paragraph, top_n=5):
    # Tokenize the paragraph into words
    words = word_tokenize(paragraph)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)
    words = [word.lower() for word in words if word.lower() not in stop_words and word not in punctuation]

    # Calculate word frequency distribution
    fdist = FreqDist(words)

    # Get the most common words as keywords
    top_keywords = fdist.most_common(top_n)

    return [keyword for keyword, _ in top_keywords]

# File Creator

In [ ]:
print(len(Allposts))